In [1]:
#! /usr/bin/env python
"""
Resample debris thickness data to enable regional stats to be computed
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False
verbose=False

In [2]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return average, variance**0.5


def weighted_percentile(sorted_list, weights, percentile):
    """
    Calculate weighted percentile of a sorted list
    """
    weights_cumsum_norm_high = np.cumsum(weights) / np.sum(weights)
#     print(weights_cumsum_norm_high)
    weights_norm = weights / np.sum(weights)
    weights_cumsum_norm_low = weights_cumsum_norm_high - weights_norm
#     print(weights_cumsum_norm_low)
    
    percentile_idx_high = np.where(weights_cumsum_norm_high >= percentile)[0][0]
#     print(percentile_idx_high)
    percentile_idx_low = np.where(weights_cumsum_norm_low <= percentile)[0][-1]
#     print(percentile_idx_low)
    
    if percentile_idx_low == percentile_idx_high:
        value_percentile = sorted_list[percentile_idx_low]
    else:
        value_percentile = np.mean([sorted_list[percentile_idx_low], sorted_list[percentile_idx_high]])

    return value_percentile


def pickle_data(fn, data):
    """Pickle data
    
    Parameters
    ----------
    fn : str
        filename including filepath
    data : list, etc.
        data to be pickled
    
    Returns
    -------
    .pkl file
        saves .pkl file of the data
    """
    with open(fn, 'wb') as f:
        pickle.dump(data, f)

#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [3]:
# Glaciers optimized
overwrite = False
rois = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14', '15', '16','17','18']
# rois = ['15']

# Percentiles
percentiles = [0.025, 0.05, 0.16, 0.25, 0.5, 0.75, 0.84, 0.95, 0.975]

# Uncertainty dataframe and dictionary for bounds
hd_uncertainty_fullfn = debris_prms.output_fp + 'hd_uncertainty_bnds-1std.csv'
hd_uncertainty_df = pd.read_csv(hd_uncertainty_fullfn)
hd_uncertainty_dict_low = dict(zip([int(np.round(x*100)) for x in hd_uncertainty_df['hd_m']], 
                                   list(hd_uncertainty_df['hd_bndlow_both'].values)))
hd_uncertainty_dict_low[0] = 0
hd_uncertainty_dict_low[1] = 0
hd_uncertainty_dict_high = dict(zip([int(np.round(x*100)) for x in hd_uncertainty_df['hd_m']], 
                                   list(hd_uncertainty_df['hd_bndhigh_both'].values)))
hd_uncertainty_dict_high[0] = hd_uncertainty_df.loc[0,'hd_bndhigh_both']
hd_uncertainty_dict_high[1] = hd_uncertainty_df.loc[0,'hd_bndhigh_both']

# Regional stats dataframe
reg_stats_fullfn = debris_prms.output_fp + 'reg_stats_hd_mf-noenhancement.csv'
reg_stats_cns = ['roi', 'dc_area_km2', 'dc_area_km2_lt_10cm', 'dc_area_km2_lt_50cm', 'dc_area_km2_lt_1m',
                 'hd_mean', 'hd_std', 
                 'hd_025', 'hd_05', 'hd_16', 'hd_25', 'hd_med', 'hd_75', 'hd_84', 'hd_95', 'hd_975',
                 'hd_low_mean', 'hd_low_std', 
                 'hd_low_025', 'hd_low_05', 'hd_low_16', 'hd_low_25', 'hd_low_med', 'hd_low_75', 'hd_low_84', 'hd_low_95', 'hd_low_975',
                 'hd_high_mean', 'hd_high_std', 
                 'hd_high_025', 'hd_high_05', 'hd_high_16', 'hd_high_25', 'hd_high_med', 'hd_high_75', 'hd_high_84', 'hd_high_95', 'hd_high_975',
                 'mf_mean', 'mf_std', 
                 'mf_025', 'mf_05', 'mf_16', 'mf_25', 'mf_med', 'mf_75', 'mf_84', 'mf_95', 'mf_975',
                 'mf_low_mean', 'mf_low_std', 
                 'mf_low_025', 'mf_low_05', 'mf_low_16', 'mf_low_25', 'mf_low_med', 'mf_low_75', 'mf_low_84', 'mf_low_95', 'mf_low_975',
                 'mf_high_mean', 'mf_high_std', 
                 'mf_high_025', 'mf_high_05', 'mf_high_16', 'mf_high_25', 'mf_high_med', 'mf_high_75', 'mf_high_84', 'mf_high_95', 'mf_high_975']
reg_stats_df = pd.DataFrame(np.zeros((len(rois)+1,len(reg_stats_cns))), columns=reg_stats_cns)


## ===== REGIONAL MELT FACTOR STATISTICS =====
hd_list_all_global = []
hd_list_all_low_global = []
hd_list_all_high_global = []
mf_list_all_global = []
mf_list_all_low_global = []
mf_list_all_high_global = []
area_m2_list_all_global = []
for nroi, roi in enumerate(rois):
        
    print('roi:', roi)
    
    # Load file if it already exists
    list_fp = debris_prms.output_fp + 'pickle_datasets/'
    if not os.path.exists(list_fp):
        os.makedirs(list_fp)
    hd_list_all_fullfn = list_fp + roi + '_hd_list_all.pkl'
    mf_list_all_fullfn = list_fp + roi + '_mf_list_all.pkl'
    area_m2_list_all_fullfn = list_fp + roi + '_area_m2_list_all.pkl'
    

    if os.path.exists(hd_list_all_fullfn.replace('.pkl','_low.pkl')) and not overwrite:
        # Debris thickness
        with open(hd_list_all_fullfn, 'rb') as f:
            hd_list_all = pickle.load(f)
        with open(hd_list_all_fullfn.replace('.pkl','_low.pkl'), 'rb') as f:
            hd_list_all_low = pickle.load(f)
        with open(hd_list_all_fullfn.replace('.pkl','_high.pkl'), 'rb') as f:
            hd_list_all_high = pickle.load(f)
        # Melt factor
        with open(mf_list_all_fullfn, 'rb') as f:
            mf_list_all = pickle.load(f)
        with open(mf_list_all_fullfn.replace('.pkl','_low.pkl'), 'rb') as f:
            mf_list_all_low = pickle.load(f)
        with open(mf_list_all_fullfn.replace('.pkl','_high.pkl'), 'rb') as f:
            mf_list_all_high = pickle.load(f)
        # Area
        with open(area_m2_list_all_fullfn, 'rb') as f:
            area_m2_list_all = pickle.load(f)
    
    # Modify the list so no enhancement factors
    mf_list_all = [1 if x>1 else x for x in mf_list_all]
    mf_list_all_low = [1 if x>1 else x for x in mf_list_all_low]
    mf_list_all_high = [1 if x>1 else x for x in mf_list_all_high]
    
    # Aggregate global data
    hd_list_all_global.extend(hd_list_all)
    hd_list_all_low_global.extend(hd_list_all_low)
    hd_list_all_high_global.extend(hd_list_all_high)
    mf_list_all_global.extend(mf_list_all)
    mf_list_all_low_global.extend(mf_list_all_low)
    mf_list_all_high_global.extend(mf_list_all_high)
    area_m2_list_all_global.extend(area_m2_list_all)
    
    def reg_stats_weighted_fromlist(list_all, area_m2_list_all, percentiles, print_name=None):
        """ Compute weighted regional stats based on list of debris thickness or melt factors and area"""
        # Sort for weighted statistics
        sorted_area_m2 = [x for _,x in sorted(zip(list_all, area_m2_list_all))]
        sorted_list = sorted(list_all)
        
        # Regional statistics
        list_mean, list_std = weighted_avg_and_std(sorted_list, weights=sorted_area_m2)
        if print_name is not None:
            print(print_name + '_mean (+/- std): ' + str(np.round(list_mean,2)) + ' +/- ' + str(np.round(list_std,2)))
        reg_stats_values = []
        reg_stats_values.append(list_mean)
        reg_stats_values.append(list_std)
        for percentile in percentiles:
            value_percentile = weighted_percentile(sorted_list, sorted_area_m2, percentile)
            reg_stats_values.append(value_percentile)
            print('  ' + print_name + ' percentile (' + str(percentile) + '): ' +  str(np.round(value_percentile,2)))
        return reg_stats_values
    
    # Compute regional statistics
    reg_stats_values = [roi, np.sum(area_m2_list_all) / 1e6]
    # ----- Debris-covered area for various thresholds -----
    hd_list_all_array = np.array(hd_list_all)
    area_km2_list_all_array = np.array(area_m2_list_all) / 1e6
    for hd_threshold in [0.1, 0.5, 1.]:
        hd_idxs = np.where(hd_list_all_array < hd_threshold)[0]
        if len(hd_idxs) > 0:
            dc_area_km2_lt_threshold = area_km2_list_all_array[hd_idxs].sum()
        else:
            dc_area_km2_lt_threshold = 0 
        reg_stats_values.extend([dc_area_km2_lt_threshold])
    # ----- Debris thickness -----
    reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all, area_m2_list_all, percentiles, print_name='hd')
    reg_stats_values.extend(reg_stats_subset)
    # Debris thickness (low uncertainty)
    reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all_low, area_m2_list_all, percentiles, print_name='hd_low')
    reg_stats_values.extend(reg_stats_subset)
    # Debris thickness (high uncertainty)
    reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all_high, area_m2_list_all, percentiles, print_name='hd_high')
    reg_stats_values.extend(reg_stats_subset)
    # ----- Melt factor -----
    reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all, area_m2_list_all, percentiles, print_name='mf')
    reg_stats_values.extend(reg_stats_subset)
    # Melt factor (low uncertainty)
    reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all_low, area_m2_list_all, percentiles, print_name='mf_low')
    reg_stats_values.extend(reg_stats_subset)
    # Melt factor
    reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all_high, area_m2_list_all, percentiles, print_name='mf_high')
    reg_stats_values.extend(reg_stats_subset)
    
    # Record regional stats
    reg_stats_df.loc[nroi,:] = reg_stats_values
    
#     print(reg_stats_values)
    print('\n')
    
# GLOBAL STATISTICS
nroi += 1
hd_list_all = hd_list_all_global
hd_list_all_low = hd_list_all_low_global
hd_list_all_high = hd_list_all_high_global
mf_list_all = mf_list_all_global
mf_list_all_low = mf_list_all_low_global
mf_list_all_high = mf_list_all_high_global
area_m2_list_all = area_m2_list_all_global

# Compute regional statistics
reg_stats_values = ['all', np.sum(area_m2_list_all) / 1e6]
# ----- Debris-covered area for various thresholds -----
hd_list_all_array = np.array(hd_list_all)
area_km2_list_all_array = np.array(area_m2_list_all) / 1e6
for hd_threshold in [0.1, 0.5, 1.]:
    hd_idxs = np.where(hd_list_all_array < hd_threshold)[0]
    if len(hd_idxs) > 0:
        dc_area_km2_lt_threshold = area_km2_list_all_array[hd_idxs].sum()
    else:
        dc_area_km2_lt_threshold = 0 
    reg_stats_values.extend([dc_area_km2_lt_threshold])
# ----- Debris thickness -----
reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all, area_m2_list_all, percentiles, print_name='hd')
reg_stats_values.extend(reg_stats_subset)
# Debris thickness (low uncertainty)
reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all_low, area_m2_list_all, percentiles, print_name='hd_low')
reg_stats_values.extend(reg_stats_subset)
# Debris thickness (high uncertainty)
reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all_high, area_m2_list_all, percentiles, print_name='hd_high')
reg_stats_values.extend(reg_stats_subset)
# ----- Melt factor -----
reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all, area_m2_list_all, percentiles, print_name='mf')
reg_stats_values.extend(reg_stats_subset)
# Melt factor (low uncertainty)
reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all_low, area_m2_list_all, percentiles, print_name='mf_low')
reg_stats_values.extend(reg_stats_subset)
# Melt factor
reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all_high, area_m2_list_all, percentiles, print_name='mf_high')
reg_stats_values.extend(reg_stats_subset)

reg_stats_df.loc[nroi,:] = reg_stats_values

# ==== Export regional stats =====
reg_stats_df.to_csv(reg_stats_fullfn, index=False)

roi: 01
hd_mean (+/- std): 0.4 +/- 0.61
  hd percentile (0.025): 0.0
  hd percentile (0.05): 0.0
  hd percentile (0.16): 0.03
  hd percentile (0.25): 0.05
  hd percentile (0.5): 0.18
  hd percentile (0.75): 0.46
  hd percentile (0.84): 0.66
  hd percentile (0.95): 1.73
  hd percentile (0.975): 3.0
hd_low_mean (+/- std): 0.24 +/- 0.33
  hd_low percentile (0.025): 0
  hd_low percentile (0.05): 0
  hd_low percentile (0.16): 0.01
  hd_low percentile (0.25): 0.03
  hd_low percentile (0.5): 0.12
  hd_low percentile (0.75): 0.3
  hd_low percentile (0.84): 0.42
  hd_low percentile (0.95): 1.01
  hd_low percentile (0.975): 1.51
hd_high_mean (+/- std): 0.7 +/- 1.41
  hd_high percentile (0.025): 0.04
  hd_high percentile (0.05): 0.04
  hd_high percentile (0.16): 0.06
  hd_high percentile (0.25): 0.08
  hd_high percentile (0.5): 0.25
  hd_high percentile (0.75): 0.63
  hd_high percentile (0.84): 0.92
  hd_high percentile (0.95): 2.87
  hd_high percentile (0.975): 7.66
mf_mean (+/- std): 0.53 +/- 0

  hd_low percentile (0.84): 0.22
  hd_low percentile (0.95): 0.5
  hd_low percentile (0.975): 0.79
hd_high_mean (+/- std): 0.36 +/- 0.89
  hd_high percentile (0.025): 0.04
  hd_high percentile (0.05): 0.04
  hd_high percentile (0.16): 0.04
  hd_high percentile (0.25): 0.06
  hd_high percentile (0.5): 0.15
  hd_high percentile (0.75): 0.31
  hd_high percentile (0.84): 0.45
  hd_high percentile (0.95): 1.11
  hd_high percentile (0.975): 1.97
mf_mean (+/- std): 0.7 +/- 0.31
  mf percentile (0.025): 0.09
  mf percentile (0.05): 0.15
  mf percentile (0.16): 0.31
  mf percentile (0.25): 0.43
  mf percentile (0.5): 0.79
  mf percentile (0.75): 1
  mf percentile (0.84): 1
  mf percentile (0.95): 1
  mf percentile (0.975): 1
mf_low_mean (+/- std): 0.8 +/- 0.28
  mf_low percentile (0.025): 0.15
  mf_low percentile (0.05): 0.22
  mf_low percentile (0.16): 0.44
  mf_low percentile (0.25): 0.6
  mf_low percentile (0.5): 1
  mf_low percentile (0.75): 1
  mf_low percentile (0.84): 1
  mf_low percenti

  mf percentile (0.975): 1
mf_low_mean (+/- std): 0.77 +/- 0.32
  mf_low percentile (0.025): 0.09
  mf_low percentile (0.05): 0.14
  mf_low percentile (0.16): 0.33
  mf_low percentile (0.25): 0.51
  mf_low percentile (0.5): 0.99
  mf_low percentile (0.75): 1
  mf_low percentile (0.84): 1.0
  mf_low percentile (0.95): 1
  mf_low percentile (0.975): 1.0
mf_high_mean (+/- std): 0.52 +/- 0.28
  mf_high percentile (0.025): 0.04
  mf_high percentile (0.05): 0.07
  mf_high percentile (0.16): 0.17
  mf_high percentile (0.25): 0.26
  mf_high percentile (0.5): 0.55
  mf_high percentile (0.75): 0.77
  mf_high percentile (0.84): 0.84
  mf_high percentile (0.95): 0.97
  mf_high percentile (0.975): 1


roi: 10
hd_mean (+/- std): 0.09 +/- 0.17
  hd percentile (0.025): 0
  hd percentile (0.05): 0
  hd percentile (0.16): 0
  hd percentile (0.25): 0.0
  hd percentile (0.5): 0.04
  hd percentile (0.75): 0.12
  hd percentile (0.84): 0.17
  hd percentile (0.95): 0.36
  hd percentile (0.975): 0.53
hd_low_me

  mf_high percentile (0.5): 0.33
  mf_high percentile (0.75): 0.72
  mf_high percentile (0.84): 0.99
  mf_high percentile (0.95): 1
  mf_high percentile (0.975): 1


roi: 14
hd_mean (+/- std): 0.36 +/- 0.57
  hd percentile (0.025): 0.01
  hd percentile (0.05): 0.01
  hd percentile (0.16): 0.04
  hd percentile (0.25): 0.06
  hd percentile (0.5): 0.16
  hd percentile (0.75): 0.4
  hd percentile (0.84): 0.59
  hd percentile (0.95): 1.4
  hd percentile (0.975): 2.83
hd_low_mean (+/- std): 0.21 +/- 0.31
  hd_low percentile (0.025): 0
  hd_low percentile (0.05): 0
  hd_low percentile (0.16): 0.02
  hd_low percentile (0.25): 0.03
  hd_low percentile (0.5): 0.1
  hd_low percentile (0.75): 0.26
  hd_low percentile (0.84): 0.38
  hd_low percentile (0.95): 0.85
  hd_low percentile (0.975): 1.45
hd_high_mean (+/- std): 0.62 +/- 1.29
  hd_high percentile (0.025): 0.04
  hd_high percentile (0.05): 0.04
  hd_high percentile (0.16): 0.07
  hd_high percentile (0.25): 0.09
  hd_high percentile (0.5): 0.

  hd_low percentile (0.16): 0.02
  hd_low percentile (0.25): 0.03
  hd_low percentile (0.5): 0.11
  hd_low percentile (0.75): 0.24
  hd_low percentile (0.84): 0.32
  hd_low percentile (0.95): 0.59
  hd_low percentile (0.975): 0.81
hd_high_mean (+/- std): 0.45 +/- 0.82
  hd_high percentile (0.025): 0.04
  hd_high percentile (0.05): 0.04
  hd_high percentile (0.16): 0.07
  hd_high percentile (0.25): 0.09
  hd_high percentile (0.5): 0.24
  hd_high percentile (0.75): 0.51
  hd_high percentile (0.84): 0.69
  hd_high percentile (0.95): 1.35
  hd_high percentile (0.975): 2.04
mf_mean (+/- std): 0.41 +/- 0.3
  mf percentile (0.025): 0.05
  mf percentile (0.05): 0.07
  mf percentile (0.16): 0.12
  mf percentile (0.25): 0.16
  mf percentile (0.5): 0.31
  mf percentile (0.75): 0.61
  mf percentile (0.84): 0.79
  mf percentile (0.95): 0.99
  mf percentile (0.975): 1
mf_low_mean (+/- std): 0.52 +/- 0.32
  mf_low percentile (0.025): 0.08
  mf_low percentile (0.05): 0.1
  mf_low percentile (0.16): 0.

In [4]:
print('\nDONE!\n')


DONE!



In [6]:
# Compute regional statistics
reg_stats_values = ['all', np.sum(area_m2_list_all) / 1e6]
# ----- Debris-covered area for various thresholds -----
hd_list_all_array = np.array(hd_list_all)
area_km2_list_all_array = np.array(area_m2_list_all) / 1e6
for hd_threshold in [0.05, 0.1, 0.25, 10.]:
    hd_idxs = np.where(hd_list_all_array < hd_threshold)[0]
    if len(hd_idxs) > 0:
        dc_area_km2_lt_threshold = area_km2_list_all_array[hd_idxs].sum()
    else:
        dc_area_km2_lt_threshold = 0 
    reg_stats_values.extend([dc_area_km2_lt_threshold])
# ----- Debris thickness -----
reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all, area_m2_list_all, percentiles, print_name='hd')
reg_stats_values.extend(reg_stats_subset)
# Debris thickness (low uncertainty)
reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all_low, area_m2_list_all, percentiles, print_name='hd_low')
reg_stats_values.extend(reg_stats_subset)
# Debris thickness (high uncertainty)
reg_stats_subset = reg_stats_weighted_fromlist(hd_list_all_high, area_m2_list_all, percentiles, print_name='hd_high')
reg_stats_values.extend(reg_stats_subset)
# ----- Melt factor -----
reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all, area_m2_list_all, percentiles, print_name='mf')
reg_stats_values.extend(reg_stats_subset)
# Melt factor (low uncertainty)
reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all_low, area_m2_list_all, percentiles, print_name='mf_low')
reg_stats_values.extend(reg_stats_subset)
# Melt factor
reg_stats_subset = reg_stats_weighted_fromlist(mf_list_all_high, area_m2_list_all, percentiles, print_name='mf_high')
reg_stats_values.extend(reg_stats_subset)

reg_stats_df.loc[nroi,:] = reg_stats_values

hd_mean (+/- std): 0.35 +/- 0.58
  hd percentile (0.025): 0.0
  hd percentile (0.05): 0.0
  hd percentile (0.16): 0.03
  hd percentile (0.25): 0.05
  hd percentile (0.5): 0.15
  hd percentile (0.75): 0.38
  hd percentile (0.84): 0.57
  hd percentile (0.95): 1.51
  hd percentile (0.975): 3.0
hd_low_mean (+/- std): 0.21 +/- 0.31
  hd_low percentile (0.025): 0
  hd_low percentile (0.05): 0
  hd_low percentile (0.16): 0.01
  hd_low percentile (0.25): 0.03
  hd_low percentile (0.5): 0.09
  hd_low percentile (0.75): 0.25
  hd_low percentile (0.84): 0.37
  hd_low percentile (0.95): 0.9
  hd_low percentile (0.975): 1.51
hd_high_mean (+/- std): 0.62 +/- 1.35
  hd_high percentile (0.025): 0.04
  hd_high percentile (0.05): 0.04
  hd_high percentile (0.16): 0.06
  hd_high percentile (0.25): 0.08
  hd_high percentile (0.5): 0.21
  hd_high percentile (0.75): 0.52
  hd_high percentile (0.84): 0.79
  hd_high percentile (0.95): 2.38
  hd_high percentile (0.975): 7.66
mf_mean (+/- std): 0.56 +/- 0.35
  

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
# Compute regional statistics
# ----- Debris-covered area for various thresholds -----
hd_list_all_array = np.array(hd_list_all)
area_km2_list_all_array = np.array(area_m2_list_all) / 1e6
for hd_threshold in [0.05, 0.1, 0.25, 10.]:
    hd_idxs = np.where(hd_list_all_array < hd_threshold)[0]
    if len(hd_idxs) > 0:
        dc_area_km2_lt_threshold = area_km2_list_all_array[hd_idxs].sum()
    else:
        dc_area_km2_lt_threshold = 0 
    reg_stats_values.extend([dc_area_km2_lt_threshold])

In [ ]:
reg_stats_cns = ['roi', 'dc_area_km2', 'dc_area_km2_lt_10cm', 'dc_area_km2_lt_50cm', 'dc_area_km2_lt_1m',
                 'hd_mean', 'hd_std', 
                 'hd_025', 'hd_05', 'hd_16', 'hd_25', 'hd_med', 'hd_75', 'hd_84', 'hd_95', 'hd_975',
                 'hd_low_mean', 'hd_low_std', 
                 'hd_low_025', 'hd_low_05', 'hd_low_16', 'hd_low_25', 'hd_low_med', 'hd_low_75', 'hd_low_84', 'hd_low_95', 'hd_low_975',
                 'hd_high_mean', 'hd_high_std', 
                 'hd_high_025', 'hd_high_05', 'hd_high_16', 'hd_high_25', 'hd_high_med', 'hd_high_75', 'hd_high_84', 'hd_high_95', 'hd_high_975',
                 'mf_mean', 'mf_std', 
                 'mf_025', 'mf_05', 'mf_16', 'mf_25', 'mf_med', 'mf_75', 'mf_84', 'mf_95', 'mf_975',
                 'mf_low_mean', 'mf_low_std', 
                 'mf_low_025', 'mf_low_05', 'mf_low_16', 'mf_low_25', 'mf_low_med', 'mf_low_75', 'mf_low_84', 'mf_low_95', 'mf_low_975',
                 'mf_high_mean', 'mf_high_std', 
                 'mf_high_025', 'mf_high_05', 'mf_high_16', 'mf_high_25', 'mf_high_med', 'mf_high_75', 'mf_high_84', 'mf_high_95', 'mf_high_975']

In [8]:
reg_stats_values

['all',
 20728.853686,
 5170.771177999994,
 8091.592554999998,
 13332.339185000006,
 20728.853686,
 0.3541929063943466,
 0.5840824947168014,
 0.0005640790256013598,
 0.00474894,
 0.026328485,
 0.05016491056521967,
 0.15109812,
 0.37667733,
 0.56642413,
 1.5124509,
 3.0,
 0.20925301864031207,
 0.31376286367083916,
 0,
 0,
 0.011398213055937377,
 0.025201296797115292,
 0.09444362901814997,
 0.24811300854399665,
 0.3682210979679148,
 0.9021522359498438,
 1.508215275979304,
 0.6197028678674389,
 1.345401416114766,
 0.04237107323823715,
 0.04237107323823715,
 0.055408889059397545,
 0.08135260662296101,
 0.2125900042298113,
 0.523685285488409,
 0.7883545751446999,
 2.378660716857952,
 7.662837607968023,
 0.5568677756398521,
 0.3531255624643325,
 0.03629816736347904,
 0.05383724,
 0.14750597,
 0.22689852830346197,
 0.5128103095798439,
 1,
 1,
 1,
 1,
 0.6550567477661714,
 0.3425370882048443,
 0.06611673888704513,
 0.08907598694095421,
 0.22021123757811795,
 0.33057894735608184,
 0.72119857050